In [ ]:
import pandas as pd
import numpy as np
import wikipedia as wiki
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
df = pd.read_csv("wiki-actors.csv")

In [ ]:
del df['Position']
df.head()

In [ ]:
def get_age(Name): 
    url = wiki.page(Name).url
    
    if not url:
        return Name + " url not found"
    
    actor_url = requests.get(url).text
    soup = BeautifulSoup(actor_url,'lxml')
    actor_table = soup.find("table",{"class":"infobox biography vcard"})
    
    if not actor_table:
        actor_table = soup.find("table",{"class":"infobox vcard"})
        if not actor_table:
            return Name + " not in infobox vcard"
    
    # Check is person has died (age will be in the "Died" section)
    p = re.compile("Died")
    match = p.search(actor_table.text)
    
    if match == None:
        age = actor_table.find("span",{"class":"noprint ForceAgeToShow"}).text
        age = age.replace(u'\xa0', u' ').replace(' (age ', "").replace(")","")
    else:       
        # age_match = re.search('(?<=;)\w+', 'aged&nbsp;88')
        # age_match = re.search('(?<= )\w+', 'aged 88')
        # age = age_match.group(0)
        
        td = soup.find(text='Died').findNext('td').text
        aged = td.replace(u'\xa0', u' ').replace(' (age ', "").replace(")","")
        match = re.compile("[0-9][0-9][a-zA-Z]").search(aged).group(0)
        # print(aged)
        age = match[:2]
    
    return int(age)

In [ ]:
for name in df['Name']:
    print(get_age(name))


In [ ]:
def get_net_worth(Name): 
    url = wiki.page(Name).url
    
    if not url:
        return Name + " url not found"
    
    actor_url = requests.get(url).text
    soup = BeautifulSoup(actor_url,'lxml')
    actor_table = soup.find("table",{"class":"infobox biography vcard"})
    
    if not actor_table:
        return Name + " not in infobox vcard"
    
    p = re.compile("Net worth") 
    match = p.search(actor_table.text)
    if match: # used the same format as 'Died' because inspect revealed similar pattern 
        td = soup.find(text='Net worth').findNext('td').text
        return td
    else:
        return "Not in vcard"
    
        
    
    

In [ ]:
def get_children_no(Name): 
     url = wiki.page(Name).url
    
    if not url:
        return Name + " url not found"
    
    actor_url = requests.get(url).text
    soup = BeautifulSoup(actor_url,'lxml')
    actor_table = soup.find("table",{"class":"infobox biography vcard"})
    
    if not actor_table:
        actor_table = soup.find("table",{"class":"infobox vcard"})
        if not actor_table:
            return Name + " not in infobox vcard"
            p = re.compile("Children") 
            match = p.search(actor_table.text)
            if match: # used the same format as 'Died' because inspect revealed similar pattern 
                td = soup.find(text='Children').findNext('td').text
                return td
            else:
                return "Not in vcard"
        
    